In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
torch.rand(10).cuda()
#conda install pytorch==1.10.0 torchvision==0.11.0 torchaudio==0.10.0 cudatoolkit=11.3 -c pytorch -c conda-forge
#pip install mayavi
#install cuda 11.3 version
#python setup.py develop
#conda install opencv
#pip install spconv-cu113
#pip install open3d filterpy


tensor([0.2039, 0.9038, 0.3394, 0.4399, 0.1318, 0.1608, 0.3738, 0.5922, 0.5319,
        0.8293], device='cuda:0')

In [3]:
#conda install numba
import argparse
import glob
from pathlib import Path

try:
    import open3d
    from visual_utils import open3d_vis_utils as V
    OPEN3D_FLAG = True
except:
    import mayavi.mlab as mlab
    from visual_utils import visualize_utils as V
    OPEN3D_FLAG = False

import numpy as np
import torch

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:


class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=True, root_path=None, logger=None, ext='.bin'):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]

        data_file_list.sort()
        self.sample_file_list = data_file_list

    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):
        if self.ext == '.bin':
            points = np.fromfile(self.sample_file_list[index], dtype=np.float32).reshape(-1, 4)
        elif self.ext == '.npy':
            points = np.load(self.sample_file_list[index])
        else:
            raise NotImplementedError

        input_dict = {
            'points': points,
            'frame_id': index,
        }

        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict


In [5]:
def parse_config():
    parser = argparse.ArgumentParser(description='arg parser')
    parser.add_argument('--cfg_file', type=str, default='cfgs/kitti_models/pv_rcnn.yaml',
                        help='specify the config for demo')
    parser.add_argument('--data_path', type=str, default='demo_data',
                        help='specify the point cloud data file or directory')
    parser.add_argument('--ckpt', default = "checkpoints/pv_rcnn_8369.pth",type=str,  help='specify the pretrained model')
    parser.add_argument('--ext', type=str, default='.bin', help='specify the extension of your point cloud data file')

    args = parser.parse_args(args=[])

    cfg_from_yaml_file(args.cfg_file, cfg)

    return args, cfg


In [6]:
args, cfg = parse_config()
logger = common_utils.create_logger()
logger.info('-----------------Quick Demo of OpenPCDet-------------------------')

2023-02-06 15:52:57,465   INFO  -----------------Quick Demo of OpenPCDet-------------------------


In [7]:
demo_dataset = DemoDataset(
    dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
    root_path=Path(args.data_path), ext=args.ext, logger=logger
)
# logger.info(f'Total number of samples: \t{len(demo_dataset)}')


In [8]:
#only car 
def apply_thres(pred_dicts,thres=0.5):
    detected_bbox_list = []
    label_list =[]
    ref_scores_list = []
    for idx,bbox in enumerate(pred_dicts[0]['pred_boxes']):
        if thres > pred_dicts[0]['pred_scores'][idx] or pred_dicts[0]['pred_labels'][idx] != 1:
            continue
        detected_bbox_list.append(bbox.tolist())
        ref_scores_list.append(pred_dicts[0]['pred_scores'][idx])
        label_list.append(pred_dicts[0]['pred_labels'][idx])
    return detected_bbox_list,ref_scores_list,label_list
# print(detected_bbox)

In [9]:
#TODO : use mayavi
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)
model.load_params_from_file(filename=args.ckpt, logger=logger, to_cpu=True)
model.cuda()
model.eval()
pred_det_list=[]
points_list=[]
ref_scores_list=[]
ref_label_list = []

with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        points_list.append(data_dict['points'][:, 1:])
        pred_dicts, _ = model.forward(data_dict)
        filtered_det,score,label = apply_thres(pred_dicts)
        pred_det_list.append(filtered_det)
        ref_scores_list.append(score)
        ref_label_list.append(label)


/home/server-003/anaconda3/envs/pcdet/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068694/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2023-02-06 15:52:57,636   INFO  ==> Loading parameters from checkpoint checkpoints/pv_rcnn_8369.pth to CPU
2023-02-06 15:52:57,670   INFO  ==> Done (loaded 367/367)


In [97]:
def translate_boxes_to_open3d_instance(gt_boxes):
    """
             4-------- 6
           /|         /|
          5 -------- 3 .
          | |        | |
          . 7 -------- 1
          |/         |/
          2 -------- 0
    """
    center = gt_boxes[0:3]
    lwh = gt_boxes[3:6]
    axis_angles = np.array([0, 0, gt_boxes[6] + 1e-10])
    rot = open3d.geometry.get_rotation_matrix_from_axis_angle(axis_angles)
    box3d = open3d.geometry.OrientedBoundingBox(center, rot, lwh)

    line_set = open3d.geometry.LineSet.create_from_oriented_bounding_box(box3d)

    # import ipdb; ipdb.set_trace(context=20)
    lines = np.asarray(line_set.lines)
    lines = np.concatenate([lines, np.array([[1, 4], [7, 6]])], axis=0)

    line_set.lines = open3d.utility.Vector2iVector(lines)

    return line_set, box3d

In [98]:
# [[h,w,l,x,y,z,theta],...]
#  bbox.h, bbox.w, bbox.l, bbox.x, bbox.y, bbox.z, bbox.ry = data[:7]
"""
    7 -------- 4
   /|         /|
  6 -------- 5 .
  | |        | |
  . 3 -------- 0
  |/         |/
  2 -------- 1
Args:
    boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center

Returns:
"""

'\n    7 -------- 4\n   /|         /|\n  6 -------- 5 .\n  | |        | |\n  . 3 -------- 0\n  |/         |/\n  2 -------- 1\nArgs:\n    boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center\n\nReturns:\n'

In [99]:
import os, numpy as np, time, sys, argparse
from tracking_modules.utils import Config, get_subfolder_seq, createFolder
from tracking_modules.io import (
    load_detection,
    get_saving_dir,
    get_frame_det,
    save_results,
    save_affinity,
)
from tracking_modules.model import AB3DMOT


In [100]:
config_path = "./tracking_modules/configs/config.yml"
cfg = Config(config_path)

In [101]:
import numpy as np
import open3d as o3d
def text_3d(text, pos, direction=None, degree=0.0,density=5, font="/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf", font_size=210):
    """
    Generate a 3D text point cloud used for visualization.
    :param text: content of the text
    :param pos: 3D xyz position of the text upper left corner
    :param direction: 3D normalized direction of where the text faces
    :param degree: in plane rotation of text
    :param font: Name of the font - change it according to your system
    :param font_size: size of the font
    :return: o3d.geoemtry.PointCloud object
    """
    if direction is None:
        direction = (0, 0., 1.)

    from PIL import Image, ImageFont, ImageDraw
    from pyquaternion import Quaternion
#     font_obj = ImageFont.load_default()
    font_obj = ImageFont.truetype(font, font_size*density)

    font_dim = font_obj.getsize(text)

    img = Image.new('RGB', font_dim, color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((0, 0), text, font=font_obj, fill=(0, 0, 0))
    img = np.asarray(img)
    img_mask = img[:, :, 0] < 128
    indices = np.indices([*img.shape[0:2], 1])[:, img_mask, 0].reshape(3, -1).T

    pcd = o3d.geometry.PointCloud()
    pcd.colors = o3d.utility.Vector3dVector(img[img_mask, :].astype(float)+100 )
#     / 255.0
    pcd.points = o3d.utility.Vector3dVector(indices / 1000 /density)

    raxis = np.cross([0.0, 0.0, 1.0], direction)
    if np.linalg.norm(raxis) < 1e-6:
        raxis = (0.0, 0.0, 1.0)
    trans = (Quaternion(axis=raxis, radians=np.arccos(direction[2])) *
             Quaternion(axis=direction, degrees=degree)).transformation_matrix
    trans[0:3, 3] = np.asarray(pos)
    pcd.transform(trans)
    return pcd
# text_3d("test",[0,0,0])

In [102]:
def tracking_vis(idx,result): 
    vis = open3d.visualization.Visualizer()
    vis.create_window()
    vis.get_render_option().point_size = 1.0
    vis.get_render_option().background_color = np.zeros(3)
    axis_pcd = open3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
    vis.add_geometry(axis_pcd)
    pts = open3d.geometry.PointCloud()
    points =points_list[idx][:,:3].cpu().numpy()
    pts.points = open3d.utility.Vector3dVector(points)
    pts.colors = open3d.utility.Vector3dVector(np.ones((points.shape[0], 3)))
    vis.add_geometry(pts)
    for i in range(result.shape[0]):
        vis_result =[]
        vis_result[0:3] = result[i,3:6]
        vis_result[3:6] = result[i,0:3]
        vis_result.append(result[i,6])
        
        line_set, box3d = translate_boxes_to_open3d_instance(vis_result)
        line_set.paint_uniform_color((1,1,0))
        id_str = str(int(result[i,7]))
#         print(id_str)
#         print(id_str)
#         print(vis_result[0:3])
        text_pose=[]
        text_pose=vis_result[0:3]
        text_pose[2]+=0.7
        text = text_3d(id_str,pos=text_pose[0:3])
    
        vis.add_geometry(text)
        vis.add_geometry(line_set)
        
#         o3d.visualization.draw_geometries([pcd, chessboard_coord])

            # if score is not None:
#     corners = box3d.get_box_points()
#     vis.add_3d_label(corners[5], '%.2f' % 0.1)
    
    vis.run()
    vis.destroy_window()

In [103]:
ID_start = 1
total_time = 0.0
tracker = AB3DMOT(cfg, "Car", ID_init=ID_start)

In [104]:

for idx,pred_det in enumerate(pred_det_list):
    since = time.time()
    results, affi = tracker.track(pred_det)
#     print(results)
    total_time += time.time() - since
    results = np.squeeze(results)
    tracking_vis(idx,results)
#     if idx==1:
#         break
# h,w,l,x,y,z,theta, ID, other info, confidence



[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]


In [47]:
# len(pred_det_list)

In [19]:
# fig = visualize_pts(points_list[0][:,:3].cpu().numpy())
# fig = draw_multi_grid_range(fig, bv_range=(0, -40, 80, 40))
# # ref_corners3d = boxes_to_corners_3d([1,1,1,1,1,1,1])
# fig = draw_corners3d([1.,1.,1.,1.,1.,1.,1.], fig=fig, color=(0, 1, 0), cls=[91], max_num=100)

In [20]:
# !pip install image
# sudo fc-cache -f -v

In [21]:

chessboard_coord = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.02, origin=[0, 0, 0])
# pcd_10 = text_3d('Test-10mm', pos=[0, 0, 0.01], font_size=10, density=10)
# pcd_20 = text_3d('Test-20mm', pos=[0, 0, 0], font_size=20, density=2)
pcd = text_3d("test",pos=[0,0,0])
o3d.visualization.draw_geometries([pcd, chessboard_coord])
